In [1]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random

# Comenzamos con los datos de genero y edad

In [2]:
concatAge = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/entrega6/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/Fiuba desde 15 Abril/fiuba_2_postulantes_genero_y_edad.csv')]
                    )
concatAge.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [3]:
concatAge.drop_duplicates('idpostulante',keep = 'last', inplace = True)
concatAge.reset_index(inplace = True)
concatAge.drop(columns = ['index'],inplace = True)
concatAge.rename(columns = {'fechanacimiento':'edad'}, inplace = True)


# hoy : 30/5/2018
hoy = pd.Timestamp(DT.datetime.now())
concatAge['edad'] = pd.to_datetime(concatAge['edad'],errors = 'coerce')
concatAge['edad'] = (hoy - concatAge['edad']).astype('<m8[Y]')
concatAge.head()

,idpostulante,edad,sexo
0,eo2p,37.0,MASC
1,1d2B,42.0,MASC
2,EBO0,44.0,FEM
3,a6MKW,43.0,MASC
4,6MWd4,43.0,MASC


In [4]:
def sexoABool(sexo):
    if (sexo == 'MASC'):
        return True
    if (sexo == 'FEM'):
        return False
    return (bool(random.getrandbits(1)))

In [5]:
concatAge['sexo'] = concatAge['sexo'].apply(sexoABool)
concatAge.rename(columns = {'sexo':'sexo_masculino'}, inplace = True)
concatAge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504407 entries, 0 to 504406
Data columns (total 3 columns):
idpostulante      504407 non-null object
edad              478311 non-null float64
sexo_masculino    504407 non-null bool
dtypes: bool(1), float64(1), object(1)
memory usage: 8.2+ MB


In [6]:
concatAge['edad'].fillna(0,inplace = True)

# Pasamos a los datos sobre la educacion

In [7]:
concatEduc = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/entrega6/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')]
                      )
concatEduc.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [8]:
concatEduc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103193 entries, 0 to 397508
Data columns (total 3 columns):
idpostulante    1103193 non-null object
nombre          1103193 non-null object
estado          1103193 non-null object
dtypes: object(3)
memory usage: 33.7+ MB


In [9]:
concatEduc['nombre'].value_counts()

Universitario        404619
Secundario           382927
Terciario/Técnico    180091
Otro                  84674
Posgrado              33484
Master                16436
Doctorado               962
Name: nombre, dtype: int64

In [10]:
def estudioSuperior(nivel):
    if ((nivel=='Posgrado') or (nivel =='Doctorado') or (nivel=='Master')):
        return 1
    else:
        return 0

In [11]:
graduados = (concatEduc['estado'].astype(str)== 'Graduado').astype(int)

universitarios = (concatEduc['nombre'].astype(str) == 'Universitario').astype(int)

secundario = (concatEduc['nombre'].astype(str) == 'Secundario').astype(int)

mayor_a_universitario = concatEduc['nombre'].apply(estudioSuperior)

In [12]:
concatEduc['titulo_universitario'] = universitarios*graduados
concatEduc['titulo_secundario'] = secundario*graduados
concatEduc['titulo_superior'] = mayor_a_universitario * graduados

In [13]:
titulos = concatEduc.groupby('idpostulante').agg({'titulo_universitario':'max',
                                                   'titulo_secundario':'max',
                                                   'titulo_superior':'max'})
titulos.reset_index(inplace = True)

titulos.head()

,idpostulante,titulo_universitario,titulo_secundario,titulo_superior
0,0z5Dmrd,1,0,0
1,0z5JW1r,0,1,0
2,0z5VvGv,0,1,0
3,0zB01pE,0,0,0
4,0zB026d,0,1,0


In [14]:
usuarios = concatAge.merge(titulos, on = 'idpostulante', how = 'left')
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior
0,eo2p,37.0,True,1.0,1.0,1.0
1,1d2B,42.0,True,0.0,0.0,0.0
2,EBO0,44.0,False,1.0,0.0,1.0
3,a6MKW,43.0,True,1.0,0.0,1.0
4,6MWd4,43.0,True,1.0,0.0,0.0


In [15]:
usuarios['titulo_universitario'] = usuarios['titulo_universitario'].astype(bool)
usuarios['titulo_secundario'] = usuarios['titulo_secundario'].astype(bool)
usuarios['titulo_superior'] = usuarios['titulo_superior'].astype(bool)

In [16]:
usuarios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 6 columns):
idpostulante            504407 non-null object
edad                    504407 non-null float64
sexo_masculino          504407 non-null bool
titulo_universitario    504407 non-null bool
titulo_secundario       504407 non-null bool
titulo_superior         504407 non-null bool
dtypes: bool(4), float64(1), object(1)
memory usage: 13.5+ MB


# Pasamos a los datos sobre los avisos

In [17]:
avisos = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/entrega6/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')])


avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [18]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45969 entries, 0 to 337
Data columns (total 11 columns):
idaviso                 45969 non-null int64
idpais                  45969 non-null int64
titulo                  45969 non-null object
descripcion             45969 non-null object
nombre_zona             45969 non-null object
ciudad                  225 non-null object
mapacalle               3444 non-null object
tipo_de_trabajo         45969 non-null object
nivel_laboral           45634 non-null object
nombre_area             45969 non-null object
denominacion_empresa    45955 non-null object
dtypes: int64(2), object(9)
memory usage: 4.2+ MB


In [19]:
avisos.drop_duplicates('idaviso', keep = 'last', inplace = True)
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 11 columns):
idaviso                 25288 non-null int64
idpais                  25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
nombre_zona             25288 non-null object
ciudad                  160 non-null object
mapacalle               2070 non-null object
tipo_de_trabajo         25288 non-null object
nivel_laboral           24953 non-null object
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


In [20]:
avisos.drop(columns = ['mapacalle','ciudad','idpais'], inplace = True)


In [21]:
avisos['tipo_de_trabajo'].value_counts()

Full-time          22831
Part-time           1746
Teletrabajo          248
Por Horas            125
Pasantia             119
Temporario            96
Por Contrato          88
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

In [22]:
avisos['tipo_de_trabajo'] = (avisos['tipo_de_trabajo'].astype(str) == 'Full-time').astype(bool)

In [23]:
avisos['nombre_zona'].value_counts()

Gran Buenos Aires              23017
Capital Federal                 2183
Buenos Aires (fuera de GBA)       57
GBA Oeste                         10
La Plata                           4
Mendoza                            3
Ciudad de Mendoza                  3
Cordoba                            2
Rosario                            2
Tucuman                            1
Corrientes                         1
Santa Cruz                         1
Santa Fe                           1
Neuquen                            1
San Juan                           1
Catamarca                          1
Name: nombre_zona, dtype: int64

In [24]:
avisos.rename(columns = {'nombre_zona':'GBA', 'tipo_de_trabajo':'Full-time'},inplace = True)

In [25]:
avisos['GBA'] = (avisos['GBA'].astype(str) == 'Gran Buenos Aires').astype(bool)
avisos.head(10)

,idaviso,titulo,descripcion,GBA,Full-time,nivel_laboral,nombre_area,denominacion_empresa
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,True,True,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL
8,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,False,True,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga
19,1110185164,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,True,True,Junior,Administración,Unión Personal
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,True,True,Junior,Distribución,Asoko Tempo SA
32,1111034024,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,True,True,Senior / Semi-Senior,Comercial,JELS SRL
35,1111101289,PM. Zona Norte / Pacheco,<p>Acciona IT se encuentra en la búsqueda de P...,True,True,Senior / Semi-Senior,Liderazgo de Proyecto,ACCIONA IT
38,1111109704,"Supervisor de obra civil, eléctrico/ instrumen...",<p>Buscamos para importante empresa petrolera ...,True,True,Senior / Semi-Senior,Construcción,Hahn Solarz S.R.L.
45,1111172357,Empleado Administrativo para Tramites de Habil...,"<p style=""""><strong><em><span style="""">Brujula...",True,True,Senior / Semi-Senior,Administración,Brujula SA
46,1111174081,Gestor de Cobranzas - Telecobrador,<p><strong>En MAS ACTIVOS BPO te estamos esper...,False,False,Senior / Semi-Senior,Call Center,MAS ACTIVOS S.A.
47,1111235995,Analista de Recursos Humanos - Quilmes (Plazo...,<p>Cervecería y Maltería Quilmes es una de las...,True,True,Senior / Semi-Senior,Recursos Humanos,AB InBev - Cervecería y Maltería Quilmes


In [26]:
avisos['nivel_laboral'].value_counts()

Senior / Semi-Senior                    16975
Junior                                   4152
Otro                                     1977
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [27]:
def nivelAEntero(nivel):
    if (nivel == 'Senior / Semi-Senior'):
        return 0
    if (nivel == 'Junior'):
        return -1
    if (nivel == 'Otro'):
        return 1
    return 3

In [28]:
avisos['nivel_laboral'] = avisos['nivel_laboral'].apply(nivelAEntero)
avisos.rename(columns = {'nivel_laboral':'nivel'}, inplace = True)


In [29]:
def esChofer(titulo):
    if ('chofer' in titulo):
        return True
    if ('Chofer' in titulo):
        return True
    return False

In [30]:
avisos['chofer'] = avisos['titulo'].apply(esChofer)
avisos['chofer'].value_counts()

False    25113
True       175
Name: chofer, dtype: int64

In [31]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 9 columns):
idaviso                 25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
GBA                     25288 non-null bool
Full-time               25288 non-null bool
nivel                   25288 non-null int64
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
chofer                  25288 non-null bool
dtypes: bool(3), int64(2), object(4)
memory usage: 1.4+ MB


# Vistas

In [32]:
concatVistas = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_3_vistas.csv'),
                          pd.read_csv('data/entrega6/fiuba_3_vistas.csv'),
                          pd.read_csv('data/Fiuba desde 15 Abril/fiuba_3_vistas.csv')])

concatVistas.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [33]:
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18368813 entries, 0 to 11648229
Data columns (total 3 columns):
idAviso         int64
timestamp       object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 560.6+ MB


In [34]:
concatVistas['visita'] = True
concatVistas.rename(columns = {'idAviso':'idaviso'}, inplace = True)
concatVistas.drop(columns = 'timestamp',inplace = True)
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18368813 entries, 0 to 11648229
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
visita          bool
dtypes: bool(1), int64(1), object(1)
memory usage: 437.9+ MB


In [35]:
concatVistas.drop_duplicates(subset = ['idaviso','idpostulante'],
                             inplace = True, keep = 'first')

In [36]:
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8597915 entries, 0 to 11648221
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
visita          bool
dtypes: bool(1), int64(1), object(1)
memory usage: 205.0+ MB


# Pasamos a las postulaciones

In [37]:
postulaciones = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_4_postulaciones.csv'),
                           pd.read_csv('data/entrega6/fiuba_4_postulaciones.csv')])
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [38]:
postulaciones.drop(columns = 'fechapostulacion', inplace = True)

In [39]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8311264 entries, 0 to 4909640
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 190.2+ MB


In [40]:
postulaciones.drop_duplicates(subset = ['idaviso','idpostulante'],
                              keep = 'first',
                              inplace = True)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 4909640
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 151.1+ MB


In [41]:
postulaciones = postulaciones.merge(usuarios,on='idpostulante',how='left')
postulaciones.head()

,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior
0,1112257047,NM5M,47.0,False,False,True,False
1,1111920714,NM5M,47.0,False,False,True,False
2,1112346945,NM5M,47.0,False,False,True,False
3,1112345547,NM5M,47.0,False,False,True,False
4,1112237522,5awk,55.0,False,True,False,False


In [42]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 7 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
dtypes: bool(4), float64(1), int64(1), object(1)
memory usage: 226.7+ MB


In [43]:
columnasInutiles =['titulo','nombre_area','descripcion','denominacion_empresa']

postulaciones = postulaciones.merge(avisos.drop(columns = columnasInutiles), 
                                    on = 'idaviso',
                                    how = 'left')

postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 11 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
GBA                     object
Full-time               object
nivel                   float64
chofer                  object
dtypes: bool(4), float64(2), int64(1), object(4)
memory usage: 428.3+ MB


In [44]:
postulaciones['postulacion'] = True
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 12 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
GBA                     object
Full-time               object
nivel                   float64
chofer                  object
postulacion             bool
dtypes: bool(5), float64(2), int64(1), object(4)
memory usage: 434.6+ MB


In [45]:
postulaciones = postulaciones.merge(concatVistas,
                                    on = ['idaviso','idpostulante'],
                                    how = 'left')
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 13 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
GBA                     object
Full-time               object
nivel                   float64
chofer                  object
postulacion             bool
visita                  object
dtypes: bool(5), float64(2), int64(1), object(5)
memory usage: 484.9+ MB


In [46]:
postulaciones['visita'].fillna(False,inplace = True)
postulaciones.head()

,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,GBA,Full-time,nivel,chofer,postulacion,visita
0,1112257047,NM5M,47.0,False,False,True,False,True,False,1.0,False,True,False
1,1111920714,NM5M,47.0,False,False,True,False,True,False,0.0,False,True,False
2,1112346945,NM5M,47.0,False,False,True,False,True,True,0.0,False,True,False
3,1112345547,NM5M,47.0,False,False,True,False,True,True,0.0,False,True,False
4,1112237522,5awk,55.0,False,True,False,False,True,True,0.0,False,True,False


In [47]:
postulaciones['chofer'] = postulaciones['chofer'].astype(bool)
postulaciones['visita'] = postulaciones['visita'].astype(bool)
postulaciones['GBA'] = postulaciones['GBA'].astype(bool)
postulaciones['Full-time'] = postulaciones['Full-time'].astype(bool)

In [50]:
postulaciones.head(20)

,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,GBA,Full-time,nivel,chofer,postulacion,visita
0,1112257047,NM5M,47.0,False,False,True,False,True,False,1.0,False,True,False
1,1111920714,NM5M,47.0,False,False,True,False,True,False,0.0,False,True,False
2,1112346945,NM5M,47.0,False,False,True,False,True,True,0.0,False,True,False
3,1112345547,NM5M,47.0,False,False,True,False,True,True,0.0,False,True,False
4,1112237522,5awk,55.0,False,True,False,False,True,True,0.0,False,True,False
5,1112277633,5awk,55.0,False,True,False,False,True,True,3.0,False,True,False
6,1112302347,5awk,55.0,False,True,False,False,True,True,3.0,False,True,True
7,1112291495,5awk,55.0,False,True,False,False,False,True,0.0,False,True,False
8,1112304011,5awk,55.0,False,True,False,False,True,True,0.0,False,True,False
9,1112326289,5awk,55.0,False,True,False,False,True,True,NaN,True,True,False


In [67]:
no_postulaciones = usuarios.sample(10000)
for i in range(1,200):
    no_postulaciones = pd.concat([no_postulaciones, usuarios.sample(20000)])

In [68]:
avisos_no_postulaciones = avisos.sample(10000)
for i in range(1,200):
    avisos_no_postulaciones = pd.concat([avisos_no_postulaciones,avisos.sample(20000)])

In [69]:
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990000 entries, 12067 to 232018
Data columns (total 6 columns):
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
dtypes: bool(4), float64(1), object(1)
memory usage: 106.5+ MB


In [70]:
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior
12067,aNBvOJ,41.0,False,False,True,False
95457,aZQQqD,37.0,False,True,False,False
140213,8MrEJlD,29.0,False,True,False,False
79573,lDLAGZ3,21.0,False,False,True,False
465094,ZzPvq8,45.0,True,True,True,True


In [71]:
no_postulaciones.reset_index(inplace = True)
no_postulaciones.drop(columns = 'index', inplace = True)
no_postulaciones

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior
0,aNBvOJ,41.0,False,False,True,False
1,aZQQqD,37.0,False,True,False,False
2,8MrEJlD,29.0,False,True,False,False
3,lDLAGZ3,21.0,False,False,True,False
4,ZzPvq8,45.0,True,True,True,True
5,52Z88p,60.0,True,True,False,False
6,xkPvkXa,33.0,False,False,True,False
7,8MBl85j,26.0,False,True,True,True
8,6PZqDr,31.0,False,True,False,False
9,akbDOJ0,42.0,False,False,True,False


In [72]:
avisos_no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990000 entries, 9256 to 8510
Data columns (total 9 columns):
idaviso                 int64
titulo                  object
descripcion             object
GBA                     bool
Full-time               bool
nivel                   int64
nombre_area             object
denominacion_empresa    object
chofer                  bool
dtypes: bool(3), int64(2), object(4)
memory usage: 224.5+ MB


In [73]:
avisos_no_postulaciones.head()

,idaviso,titulo,descripcion,GBA,Full-time,nivel,nombre_area,denominacion_empresa,chofer
9256,1112114104,- Functional Tester -,<p>Cognizant is a leader provider of informati...,True,True,1,Testing / QA / QC,Cognizant Technology Solutions,False
5649,1112336561,Gestor de cobranzas,"<p align=""right""> </p><p><strong>INCORPORAMOS ...",False,False,0,Créditos y Cobranzas,RC COBRANZAS,False
6226,1112464708,Comprador Sr,"<p style="""">En <span style=""""><em style="""">Tec...",True,True,0,Compras,Techint Ingeniería y Construcción,False
7764,1112476739,Técnico Electromecánico / Electrónico / Proye...,<p>Para importante empresa de climatización de...,True,True,0,Jóvenes Profesionales,ESTUDIO SANCHEZ SALAS Y ASOCIADOS SRL,False
6600,1112470387,Personal para supermercados,"<p><span style="""">Importante cadena de superme...",True,True,0,Atención al Cliente,Grupo Gestión,False


In [81]:
avisos_no_postulaciones.reset_index(inplace = True)
avisos_no_postulaciones.drop(columns = columnasInutiles, inplace = True)

avisos_no_postulaciones.head()

,idaviso,GBA,Full-time,nivel,chofer
0,1112114104,True,True,1,False
1,1112336561,False,False,0,False
2,1112464708,True,True,0,False
3,1112476739,True,True,0,False
4,1112470387,True,True,0,False


In [82]:
avisos_no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3990000 entries, 0 to 3989999
Data columns (total 5 columns):
idaviso      int64
GBA          bool
Full-time    bool
nivel        int64
chofer       bool
dtypes: bool(3), int64(2)
memory usage: 72.3 MB


In [83]:
no_postulaciones = no_postulaciones.join(avisos_no_postulaciones)
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3990000 entries, 0 to 3989999
Data columns (total 11 columns):
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
idaviso                 int64
GBA                     bool
Full-time               bool
nivel                   int64
chofer                  bool
dtypes: bool(7), float64(1), int64(2), object(1)
memory usage: 148.4+ MB


In [84]:
no_postulaciones['postulacion'] = False
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,idaviso,GBA,Full-time,nivel,chofer,postulacion
0,aNBvOJ,41.0,False,False,True,False,1112114104,True,True,1,False,False
1,aZQQqD,37.0,False,True,False,False,1112336561,False,False,0,False,False
2,8MrEJlD,29.0,False,True,False,False,1112464708,True,True,0,False,False
3,lDLAGZ3,21.0,False,False,True,False,1112476739,True,True,0,False,False
4,ZzPvq8,45.0,True,True,True,True,1112470387,True,True,0,False,False


In [85]:
no_postulaciones = no_postulaciones.merge(concatVistas,
                                          on = ['idaviso','idpostulante'],
                                          how = 'left')
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,idaviso,GBA,Full-time,nivel,chofer,postulacion,visita
0,aNBvOJ,41.0,False,False,True,False,1112114104,True,True,1,False,False,NaN
1,aZQQqD,37.0,False,True,False,False,1112336561,False,False,0,False,False,NaN
2,8MrEJlD,29.0,False,True,False,False,1112464708,True,True,0,False,False,NaN
3,lDLAGZ3,21.0,False,False,True,False,1112476739,True,True,0,False,False,NaN
4,ZzPvq8,45.0,True,True,True,True,1112470387,True,True,0,False,False,NaN


In [86]:
no_postulaciones['visita'].fillna(False, inplace = True)
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990000 entries, 0 to 3989999
Data columns (total 13 columns):
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
idaviso                 int64
GBA                     bool
Full-time               bool
nivel                   int64
chofer                  bool
postulacion             bool
visita                  bool
dtypes: bool(9), float64(1), int64(2), object(1)
memory usage: 186.5+ MB


In [87]:
no_postulaciones = no_postulaciones[postulaciones.columns]
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3990000 entries, 0 to 3989999
Data columns (total 13 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
GBA                     bool
Full-time               bool
nivel                   int64
chofer                  bool
postulacion             bool
visita                  bool
dtypes: bool(9), float64(1), int64(2), object(1)
memory usage: 186.5+ MB


In [88]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 13 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
GBA                     bool
Full-time               bool
nivel                   float64
chofer                  bool
postulacion             bool
visita                  bool
dtypes: bool(9), float64(2), int64(1), object(1)
memory usage: 308.6+ MB


In [89]:
postulaciones = pd.concat([postulaciones,no_postulaciones])

In [90]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10593752 entries, 0 to 3989999
Data columns (total 13 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
GBA                     bool
Full-time               bool
nivel                   float64
chofer                  bool
postulacion             bool
visita                  bool
dtypes: bool(9), float64(2), int64(1), object(1)
memory usage: 495.0+ MB


In [101]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10593752 entries, 0 to 3989999
Data columns (total 13 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
GBA                     bool
Full-time               bool
nivel                   float64
chofer                  bool
postulacion             bool
visita                  bool
dtypes: bool(9), float64(2), int64(1), object(1)
memory usage: 495.0+ MB


In [102]:
postulaciones.drop_duplicates(subset = ['idaviso','idpostulante'], keep = 'first',inplace = True)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10591209 entries, 0 to 3989999
Data columns (total 13 columns):
idaviso                 int64
idpostulante            object
edad                    float64
sexo_masculino          bool
titulo_universitario    bool
titulo_secundario       bool
titulo_superior         bool
GBA                     bool
Full-time               bool
nivel                   float64
chofer                  bool
postulacion             bool
visita                  bool
dtypes: bool(9), float64(2), int64(1), object(1)
memory usage: 494.9+ MB


In [106]:
postulaciones.to_csv('training_set_1')

In [110]:
test = pd.read_csv('data/test_final_100k.csv')

In [111]:
test = test.merge(usuarios, on='idpostulante', how = 'left')
test.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior
0,0,739260,6M9ZQR,42.0,False,True,False,True
1,1,739260,6v1xdL,30.0,True,False,False,False
2,2,739260,ezRKm9,36.0,False,True,False,False
3,3,758580,1Q35ej,68.0,True,True,False,True
4,4,758580,EAN4J6,32.0,False,False,False,False


In [113]:
test = test.merge(avisos.drop(columns = columnasInutiles),
                            on = 'idaviso',
                            how = 'left')
test.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,GBA,Full-time,nivel,chofer
0,0,739260,6M9ZQR,42.0,False,True,False,True,False,True,3,False
1,1,739260,6v1xdL,30.0,True,False,False,False,False,True,3,False
2,2,739260,ezRKm9,36.0,False,True,False,False,False,True,3,False
3,3,758580,1Q35ej,68.0,True,True,False,True,False,True,1,False
4,4,758580,EAN4J6,32.0,False,False,False,False,False,True,1,False


In [114]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
id                      100000 non-null int64
idaviso                 100000 non-null int64
idpostulante            100000 non-null object
edad                    97162 non-null float64
sexo_masculino          100000 non-null bool
titulo_universitario    100000 non-null bool
titulo_secundario       100000 non-null bool
titulo_superior         100000 non-null bool
GBA                     100000 non-null bool
Full-time               100000 non-null bool
nivel                   100000 non-null int64
chofer                  100000 non-null bool
dtypes: bool(7), float64(1), int64(3), object(1)
memory usage: 5.2+ MB


In [118]:
test['edad'].fillna(0, inplace = True)

In [119]:
test = test.merge(concatVistas, on = ['idaviso','idpostulante'], how = 'left')
test.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,GBA,Full-time,nivel,chofer,visita
0,0,739260,6M9ZQR,42.0,False,True,False,True,False,True,3,False,NaN
1,1,739260,6v1xdL,30.0,True,False,False,False,False,True,3,False,NaN
2,2,739260,ezRKm9,36.0,False,True,False,False,False,True,3,False,NaN
3,3,758580,1Q35ej,68.0,True,True,False,True,False,True,1,False,NaN
4,4,758580,EAN4J6,32.0,False,False,False,False,False,True,1,False,NaN


In [120]:
test['visita'].fillna(False, inplace = True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 13 columns):
id                      100000 non-null int64
idaviso                 100000 non-null int64
idpostulante            100000 non-null object
edad                    100000 non-null float64
sexo_masculino          100000 non-null bool
titulo_universitario    100000 non-null bool
titulo_secundario       100000 non-null bool
titulo_superior         100000 non-null bool
GBA                     100000 non-null bool
Full-time               100000 non-null bool
nivel                   100000 non-null int64
chofer                  100000 non-null bool
visita                  100000 non-null bool
dtypes: bool(8), float64(1), int64(3), object(1)
memory usage: 5.3+ MB


In [145]:
postulaciones['nivel'].isnull().sum()

0

In [138]:
postulaciones['nivel'].fillna(0,inplace = True)

# Empieza perceptron


In [124]:
columnasTraining = ['edad','sexo_masculino','titulo_universitario',
                    'titulo_secundario','titulo_superior','GBA','Full-time',
                    'nivel','chofer','visita']

In [146]:
datos = postulaciones[columnasTraining]
categorias = postulaciones['postulacion']

In [147]:
test_size = 0.3
random_state = 0

In [148]:
datos_train, datos_test, cat_train, cat_test = train_test_split(datos,
                                                               categorias,
                                                               test_size = test_size,
                                                               random_state = random_state)

In [149]:
sc = StandardScaler()
sc.fit(datos_train)

datos_train_std = sc.transform(datos_train)
datos_test_std = sc.transform(datos_test)

In [150]:
data_train = datos_train_std[:,0:5]
data_test = datos_test_std[:,0:5]
n_iter=40
eta0 = 0.1

In [151]:
#ppn1 features del 0 al 5

ppn1 = Perceptron(n_iter = n_iter,
                eta0 = eta0,
                random_state = random_state)

ppn1.fit(data_train,cat_train)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      max_iter=None, n_iter=40, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [153]:
cat_pred = ppn1.predict(data_test)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 45.32%


In [154]:
#ppn2 con todos los features
ppn2 = Perceptron(n_iter = n_iter,
                eta0 = eta0,
                random_state = random_state)

ppn2.fit(datos_train_std,cat_train)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      max_iter=None, n_iter=40, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [155]:
cat_pred = ppn2.predict(datos_test_std)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 62.69%


In [156]:
final_test_data = test[columnasTraining]

In [157]:
final_test_data.head()

,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,GBA,Full-time,nivel,chofer,visita
0,42.0,False,True,False,True,False,True,3,False,False
1,30.0,True,False,False,False,False,True,3,False,False
2,36.0,False,True,False,False,False,True,3,False,False
3,68.0,True,True,False,True,False,True,1,False,False
4,32.0,False,False,False,False,False,True,1,False,False


In [158]:
final_test_data = sc.transform(final_test_data)

In [162]:
prediccion = ppn2.predict(final_test_data)
prediccion = prediccion.astype(int)
prediccion[0:40]

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0])

In [163]:
test['sepostulo'] = prediccion
entregable = test[['id','sepostulo']]

In [164]:
entregable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 2 columns):
id           100000 non-null int64
sepostulo    100000 non-null int64
dtypes: int64(2)
memory usage: 2.3 MB


In [165]:
entregable.index = entregable['id']
entregable.drop(columns = 'id', inplace = True)
entregable

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,sepostulo
id,
0,0
1,0
2,0
3,0
4,1
5,1
6,0
7,1
8,0


In [166]:
entregable.to_csv('submits/submit_2.csv')